In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
homestays_df = pd.read_csv("homestays.csv")
homestays_df.Address = homestays_df.Address.str.upper()
homestays_df.Address = homestays_df.Address.str.strip()
homestays_df.Address = homestays_df.Address.str.replace(",", "")
homestays_df.Address = homestays_df.Address.str.replace("STREET", "ST")
homestays_df.Address = homestays_df.Address.str.replace("ROAD", "RD")
homestays_df.Address = homestays_df.Address.str.replace("AVENUE", "AVE")
homestays_df.Address = homestays_df.Address.str.replace("LANE", "LN")
homestays_df.Address = homestays_df.Address.str.replace("DRIVE", "DR")
homestays_df.Address = homestays_df.Address.str.replace("PLACE", "PL")
homestays_df.Address = homestays_df.Address.str.replace("CIRCLE", "CIR")
homestays_df.Address = homestays_df.Address.str.replace("APT ", "#")

In [3]:
points_gdf = gpd.read_file("Master_Address_Points/Master_Address_Points.shp")
points_gdf.address = points_gdf.address.str.upper()
points_gdf.address = points_gdf.address.str.strip()
points_gdf.address = points_gdf.address.str.replace("STREET", "ST")
points_gdf.address = points_gdf.address.str.replace("ROAD", "RD")
points_gdf.address = points_gdf.address.str.replace("AVENUE", "AVE")
points_gdf.address = points_gdf.address.str.replace("LANE", "LN")
points_gdf.address = points_gdf.address.str.replace("DRIVE", "DR")
points_gdf.address = points_gdf.address.str.replace("PLACE", "PL")
points_gdf.address = points_gdf.address.str.replace("CIRCLE", "CIR")

In [4]:
# merge homestays and points datasets 
# note: had to manaully fix some addresses in homestays dataset that were misspelled or incorrect
homestays_points_gdf = pd.merge(homestays_df, points_gdf, left_on="Address", right_on="address")
gdf = gpd.GeoDataFrame(homestays_points_gdf, geometry="geometry", crs={'init': 'epsg:4326'})
# gdf = gdf.to_crs({'init': 'epsg:3857'}) # to meters
# gdf['geom_poly'] = gdf.geometry.buffer(20)
# gdf = gdf.set_geometry('geom_poly')
# gdf = gdf.to_crs({'init': 'epsg:4326'})
gdf = gdf[["Name", "Address", "approval date", "geometry"]]

In [5]:
airbnb_df = pd.read_csv("../scrapy-splash/airbnb/airbnb_cville.csv")
airbnb_df["geometry"] = [Point(xy) for xy in zip(airbnb_df.longitude, airbnb_df.latitude)]
airbnb_gdf = gpd.GeoDataFrame(airbnb_df, geometry="geometry", crs={'init': 'epsg:4326'})
airbnb_gdf = airbnb_gdf.to_crs({'init': 'epsg:3857'}) # to meters
airbnb_gdf['geom_poly'] = airbnb_gdf.geometry.buffer(50)
airbnb_gdf = airbnb_gdf.set_geometry('geom_poly')
airbnb_gdf = airbnb_gdf.to_crs({'init': 'epsg:4326'})
airbnb_gdf = airbnb_gdf[["accuracy", "location", "value", "geom_poly"]]

# with pd.option_context("display.max_columns", 50):
#     display(airbnb_gdf)

In [6]:
vrbo_df = pd.read_csv("../scrapy-splash/vrbo/vrbo_cville.csv")
vrbo_df = vrbo_df.dropna(subset=["latitude", "longitude"])
vrbo_df["geometry"] = [Point(xy) for xy in zip(vrbo_df.longitude, vrbo_df.latitude)]
vrbo_gdf = gpd.GeoDataFrame(vrbo_df, geometry="geometry", crs={'init': 'epsg:4326'})
vrbo_gdf = vrbo_gdf.to_crs({'init': 'epsg:3857'}) # to meters
vrbo_gdf['geom_poly'] = vrbo_gdf.geometry.buffer(50)
vrbo_gdf = vrbo_gdf.set_geometry('geom_poly')
vrbo_gdf = vrbo_gdf.to_crs({'init': 'epsg:4326'})
vrbo_gdf = vrbo_gdf[["geom_poly"]]

In [7]:
gdf.to_file("homestays.geojson", driver='GeoJSON')
airbnb_gdf.to_file("airbnb.geojson", driver='GeoJSON')
vrbo_gdf.to_file("vrbo.geojson", driver='GeoJSON')

/Users/cwhalen/anaconda3/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [8]:
with pd.option_context("display.max_columns", 50):
    display(gpd.sjoin(gdf, vrbo_gdf, how="inner", op='within'))

,Name,Address,approval date,geometry,index_right
0,"Amato, Johnathan",707 ROCK CREEK RD,5/5/17,POINT (-78.50020315453321 38.02292236748883),9
1,"Amato, Jonathan",707 ROCK CREEK RD,4/20/16,POINT (-78.50020315453321 38.02292236748883),9
162,"Lyons, Shawn",709 ROCK CREEK RD,3/30/18,POINT (-78.50002360079461 38.02282713358147),9
7,"Hasenkamp, Wendy",219 DOUGLAS AVE #A,3/28/17,POINT (-78.47459364206132 38.02572325579347),98
25,"Clayton, Signe",217 DOUGLAS AVE,11/15/16,POINT (-78.47447709185447 38.02591414213745),98
25,"Clayton, Signe",217 DOUGLAS AVE,11/15/16,POINT (-78.47447709185447 38.02591414213745),120
28,"Terrill, Lynda & Thomas",421 4TH ST NE,7/29/16,POINT (-78.477638762859 38.03368208294136),61
67,"Tharp, Steve",405 HEDGE ST,6/2/14,POINT (-78.47713669247446 38.03380334349798),61
30,"Stien, Jennifer",211 5TH ST SW #102,4/26/16,POINT (-78.48874156898873 38.03002022678946),115
43,"Millich, Frank",414 ARBOR CIR,9/9/14,POINT (-78.46305647483035 38.02883874628062),38


In [9]:
import numpy as np
tmp = gpd.sjoin(gdf, vrbo_gdf, how="inner", op='within')
len(np.unique(tmp.Address))

22